---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025


# TP : Classification vs Détection — Comprendre CNN, R-CNN et YOLO

Dans ce TP, tu vas comparer trois familles de modèles de vision artificielle :

- **CNN (classification simple)**
- **R-CNN (détection en deux étapes)**
- **YOLO (détection en une seule passe, temps réel)**

Tu vas utiliser ces modèles sur les **mêmes images**, pour comprendre :

- ce que chaque modèle **peut** faire,
- ce qu’il **ne peut pas** faire,
- **pourquoi** on utilise YOLO aujourd’hui pour la détection.

---

## 🎯 Objectifs du TP

- Comprendre la différence entre **classification** et **détection d’objets**.  
- Voir comment un CNN simple atteint ses limites pour la localisation.  
- Utiliser un modèle de type **R-CNN** pour obtenir des boîtes englobantes.  
- Utiliser **YOLO** pour comparer vitesse, efficacité et temps réel.  
- Construire une **synthèse claire** entre les trois familles de modèles.



---
# 🟦 PARTIE 0 — Préparation du Notebook

👉 Dans cette partie, on installe les bibliothèques nécessaires et on prépare l'environnement.

> **Q0.1** : Pourquoi a-t-on besoin de bibliothèques pré-entraînées pour ce TP ?  
> **Q0.2** : Quels sont les avantages d’utiliser des modèles déjà entraînés (comme ResNet ou YOLO) ?


In [ ]:

# Installation (à exécuter dans Colab ou un environnement avec internet)
# Si tu es sur Google Colab, vérifie que tu as bien un GPU (Menu : Exécution > Modifier le type d'exécution > GPU).

!pip install ultralytics -q

import torch, torchvision
print("Version torch :", torch.__version__)
print("GPU disponible :", torch.cuda.is_available())



---
# 🟦 PARTIE 1 — CNN : Classification d’une image entière

Un **CNN de classification** prend une image en entrée et renvoie **une seule étiquette** (chien, chat, voiture…).  
Il ne renvoie **aucune information de position** des objets dans l’image.

> **Q1.1** : En une phrase, explique la différence entre *classification* et *détection d’objets*.  
> **Q1.2** : Pourquoi la détection d’objets est-elle un problème plus complexe que la simple classification ?



## 1.1 Chargement d’un modèle de classification (ResNet18)

On va utiliser **ResNet18**, un réseau de neurones convolutif pré-entraîné sur le jeu de données ImageNet.


In [ ]:

import torch
from torchvision import models, transforms
from PIL import Image
from IPython.display import display

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Périphérique utilisé :", device)

# Chargement du modèle de classification
model_cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model_cnn.eval().to(device)

# Prétraitement associé au modèle
preprocess = models.ResNet18_Weights.DEFAULT.transforms()
labels = models.ResNet18_Weights.DEFAULT.meta["categories"]

print("Modèle ResNet18 chargé avec succès.")



## 1.2 Fonction de classification d’une image

On définit une fonction qui :

1. charge l’image,
2. applique les prétraitements,
3. passe l’image dans le réseau,
4. affiche les **5 classes les plus probables**.


In [ ]:

def classify_image(path):
    img = Image.open(path).convert("RGB")
    x = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model_cnn(x)
        probs = torch.softmax(logits, dim=1)[0]

    topk = torch.topk(probs, 5)
    print("Top 5 prédictions :")
    for score, idx in zip(topk.values, topk.indices):
        print(f"{labels[idx]} : {score:.3f}")

    display(img)

print("Fonction classify_image prête.")



## 1.3 Tester la classification sur une image

1. Téléverse une image contenant **un seul objet principal** (par exemple : un chien, une voiture...).  
2. Observe la prédiction du CNN.

> **Q1.3** : Que renvoie exactement le réseau (type de sortie) ?  
> **Q1.4** : Est-ce que le réseau donne une information sur la **position** de l’objet dans l’image ? Explique.  
> **Q1.5** : Quel serait le problème si l’image contenait **plusieurs objets différents** (ex : une personne + un chien + une voiture) ?


In [ ]:

# Téléversement d'une image (Google Colab)
from google.colab import files

print("Choisis une image sur ton ordinateur (format .jpg ou .png).")
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
print("Image choisie :", image_path)

classify_image(image_path)



> **Q1.6** : Explique, en quelques phrases, pourquoi un CNN de classification ne peut pas résoudre le problème de **détection d’objets**.  
> (Indice : une seule prédiction pour toute l’image, aucune coordonnée de boîte...)



---
# 🟩 PARTIE 2 — R-CNN : Détection en deux étapes

Les modèles de type **R-CNN** sont conçus pour la **détection d’objets** : ils renvoient des **boîtes englobantes** et des **classes** pour chaque objet détecté.

Idée générale (simplifiée) :  
1. Proposer des **régions candidates** (zones où il pourrait y avoir un objet),  
2. Appliquer un CNN sur chaque région,  
3. Prédire la **classe** et affiner la **boîte** pour chaque objet.

Dans ce TP, on va utiliser un modèle pratique : **Faster R-CNN** (implémenté dans `torchvision`).



## 2.1 Chargement de Faster R-CNN

On charge un modèle pré-entraîné sur le jeu de données COCO.


In [ ]:

from torchvision.models.detection import fasterrcnn_resnet50_fpn

model_rcnn = fasterrcnn_resnet50_fpn(weights="DEFAULT").to(device)
model_rcnn.eval()

print("Modèle Faster R-CNN chargé.")



## 2.2 Fonction de détection avec R-CNN

On applique le modèle sur la **même image** que dans la Partie 1 et on affiche des boîtes.


In [ ]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
from torchvision import transforms as T

def detect_rcnn(path, score_threshold=0.5):
    img = Image.open(path).convert("RGB")
    transform = T.ToTensor()
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model_rcnn(x)[0]

    boxes = outputs["boxes"].cpu().numpy()
    labels_rcnn = outputs["labels"].cpu().numpy()
    scores = outputs["scores"].cpu().numpy()

    img_np = np.array(img)

    for (box, label, score) in zip(boxes, labels_rcnn, scores):
        if score < score_threshold:
            continue
        x1, y1, x2, y2 = box.astype(int)
        cv2.rectangle(img_np, (x1, y1), (x2, y2), 2)
        cv2.putText(img_np, f"{label} ({score:.2f})", (x1, max(y1-10, 0)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2, 1)

    plt.imshow(img_np)
    plt.axis("off")

print("Fonction detect_rcnn prête.")



## 2.3 Détection sur la même image qu’en Partie 1

On applique maintenant Faster R-CNN sur la **même image** que celle utilisée avec le CNN de classification.

> **Q2.1** : Combien d’objets semblent être détectés sur l’image ?  
> **Q2.2** : Quelles informations supplémentaires obtiens-tu par rapport au CNN de la Partie 1 ?


In [ ]:

detect_rcnn(image_path, score_threshold=0.5)



> **Q2.3** : Décris la forme de la sortie de Faster R-CNN (types d'informations).  
> **Q2.4** : En quoi cette sortie est-elle fondamentalement différente de celle du CNN de classification ?  
> **Q2.5** : Pourquoi ce type de modèle est-il plus **lent** qu’un CNN simple (même si on ne voit pas forcément la différence sur une seule image) ?  
> **Q2.6** : Selon toi, quel problème peut se poser si on veut utiliser ce modèle en **temps réel** sur une webcam ?



---
# 🟧 PARTIE 3 — YOLO : Détection en une seule passe (temps réel)

Les modèles **YOLO (You Only Look Once)** sont aussi des réseaux convolutifs, mais organisés pour faire la détection en **une seule passe** sur l’image complète.

Idée clé :  
- l’image est divisée en une **grille**,  
- le réseau prédit, pour chaque zone, des **boîtes + classes**,  
- tout est calculé en une seule fois, ce qui permet la **vitesse** et le **temps réel**.

> **Q3.1** : Pourquoi l’idée de “regarder l’image une seule fois” (You Only Look Once) est-elle intéressante pour le temps réel ?



## 3.1 Chargement du modèle YOLOv8

On utilise la bibliothèque `ultralytics` pour charger un modèle YOLOv8 pré-entraîné.


In [ ]:

from ultralytics import YOLO

# Modèle YOLOv8 "small" (compromis vitesse/précision)
model_yolo = YOLO("yolov8s.pt")
print("Modèle YOLOv8s chargé.")



## 3.2 Détection avec YOLO sur la même image

On applique YOLO sur la **même image** que dans les parties 1 et 2.


In [ ]:

def detect_yolo(path):
    results = model_yolo(path)
    for r in results:
        img_plot = r.plot()  # numpy (BGR)
        img_plot = cv2.cvtColor(img_plot, cv2.COLOR_BGR2RGB)
        plt.imshow(img_plot)
        plt.axis("off")

print("Fonction detect_yolo prête.")


In [ ]:

detect_yolo(image_path)



> **Q3.2** : YOLO renvoie-t-il, lui aussi, des boîtes englobantes et des classes pour plusieurs objets ?  
> **Q3.3** : Par rapport à Faster R-CNN, que remarques-tu sur la “réactivité” du modèle (même si on ne fait qu’une image ici) ?  
> **Q3.4** : Pourquoi YOLO est-il particulièrement adapté à une utilisation **sur webcam** ou **sur Raspberry Pi** ?  
> **Q3.5** : YOLO utilise-t-il encore des convolutions comme les CNN classiques ?  



### (Optionnel) 3.3 YOLO sur la webcam

Cette partie nécessite un accès caméra (souvent plus simple en local que sur Colab).

```python
# Exemple indicatif (peut ne pas fonctionner directement sur Colab)
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model_yolo(frame)
    plotted = results[0].plot()
    cv2.imshow("YOLOv8 Webcam", plotted)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
```



---
# 🟨 PARTIE 4 — Synthèse Finale

Dans cette dernière partie, tu vas comparer les trois familles de modèles et rédiger une courte conclusion.

## 4.1 Tableau comparatif (à compléter)

Recopie et complète le tableau suivant dans ton cahier ou dans une cellule texte :

| Modèle | Tâche principale | Sortie | Avantages | Limites |
|--------|------------------|--------|-----------|---------|
| CNN    |                  |        |           |         |
| R-CNN  |                  |        |           |         |
| YOLO   |                  |        |           |         |

> **Q4.1** : Pour chaque modèle, précise s’il fait de la **classification** ou de la **détection**, et ce qu’il renvoie exactement en sortie.

## 4.2 Questions de synthèse

> **Q4.2** : Explique ce que permet un **CNN**, un **R-CNN** puis un **YOLO**, et les limites de chacun.  
> **Q4.3** : Dans quel cas précis utiliserais-tu :  
> - un CNN simple ?  
> - un modèle de type R-CNN ?  
> - un modèle de type YOLO ?  

> **Q4.4** : Si tu devais créer une application de détection d’objets **en temps réel** (par exemple pour suivre des objets avec une webcam), quel modèle choisirais-tu et pourquoi ?


👏 Bravo, tu as maintenant manipulé trois grandes familles de modèles de vision et compris leurs différences fondamentales :
- **CNN** : reconnaît *quoi* il y a dans une image.  
- **R-CNN** : reconnaît *quoi* et *où*, mais de manière plus lourde.  
- **YOLO** : reconnaît *quoi* et *où* en **temps réel**, en une seule passe.


---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025